In [4]:
%pip install plotly==5.14.1 --user



Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [6]:
path = glob('images/*.xml')
path[:]


['images\\N1.xml',
 'images\\N100.xml',
 'images\\N101.xml',
 'images\\N102.xml',
 'images\\N103.xml',
 'images\\N104.xml',
 'images\\N105.xml',
 'images\\N106.xml',
 'images\\N107.xml',
 'images\\N108.xml',
 'images\\N109.xml',
 'images\\N11.xml',
 'images\\N110.xml',
 'images\\N111.xml',
 'images\\N112.xml',
 'images\\N113.xml',
 'images\\N114.xml',
 'images\\N116.xml',
 'images\\N117.xml',
 'images\\N118.xml',
 'images\\N119.xml',
 'images\\N12.xml',
 'images\\N120.xml',
 'images\\N121.xml',
 'images\\N122.xml',
 'images\\N123.xml',
 'images\\N124.xml',
 'images\\N126.xml',
 'images\\N127.xml',
 'images\\N128.xml',
 'images\\N129.xml',
 'images\\N130.xml',
 'images\\N131.xml',
 'images\\N132.xml',
 'images\\N133.xml',
 'images\\N134.xml',
 'images\\N135.xml',
 'images\\N136.xml',
 'images\\N137.xml',
 'images\\N138.xml',
 'images\\N139.xml',
 'images\\N14.xml',
 'images\\N140.xml',
 'images\\N141.xml',
 'images\\N142.xml',
 'images\\N143.xml',
 'images\\N144.xml',
 'images\\N145.xml

In [8]:
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)


In [9]:
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [10]:
df = pd.DataFrame(labels_dict)
df.to_csv('label.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,images\N1.xml,1093,1396,645,727
1,images\N100.xml,134,301,312,350
2,images\N101.xml,31,139,128,161
3,images\N102.xml,164,316,216,243
4,images\N103.xml,813,1067,665,724


In [11]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('images/',filename_image)
    return filepath_image
getFilename(filename)

'images/N1.jpeg'

In [12]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

['images/N1.jpeg',
 'images/N100.jpeg',
 'images/N101.jpeg',
 'images/N102.jpeg',
 'images/N103.jpeg',
 'images/N104.jpeg',
 'images/N105.jpeg',
 'images/N106.jpeg',
 'images/N107.jpeg',
 'images/N108.jpeg']

In [13]:
file_path = image_path[87] #path of our image N2.jpeg
print(file_path)
img = cv2.imread(file_path) #read the image
# xmin-1804/ymin-1734/xmax-2493/ymax-1882 
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 8 - N2.jpeg with bounding box')
fig.add_shape(type='rect',x0=572, x1=839, y0=461, y1=535, xref='x', yref='y',line_color='cyan')

images/N187.jpeg


In [14]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [15]:
# Convert data to array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [16]:
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

In [17]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [18]:
# Complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [19]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=180,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/180
18/18 [==============================] - 148s 6s/step - loss: 0.0383 - val_loss: 0.0258
Epoch 2/180
18/18 [==============================] - 83s 5s/step - loss: 0.0150 - val_loss: 0.0169
Epoch 3/180
18/18 [==============================] - 91s 5s/step - loss: 0.0085 - val_loss: 0.0248
Epoch 4/180
18/18 [==============================] - 84s 5s/step - loss: 0.0066 - val_loss: 0.0122
Epoch 5/180
18/18 [==============================] - 85s 5s/step - loss: 0.0047 - val_loss: 0.0099
Epoch 6/180
18/18 [==============================] - 85s 5s/step - loss: 0.0025 - val_loss: 0.0090
Epoch 7/180
18/18 [==============================] - 80s 4s/step - loss: 0.0021 - val_loss: 0.0068
Epoch 8/180
18/18 [==============================] - 79s 4s/step - loss: 0.0017 - val_loss: 0.0061
Epoch 9/180
18/18 [==============================] - 80s 4s/step - loss: 0.0014 - val_loss: 0.0066
Epoch 10/180
18/18 [==============================] - 81s 4s/step - loss: 0.0014 - val_loss: 0.0075
Epoch 11

In [20]:
model.save('./object_detection.h5')

In [96]:
# Load model
model = tf.keras.models.load_model('./object_detection.h5')
print('Model loaded Sucessfully')

Model loaded Sucessfully


In [59]:
path = r"C:\Users\Sumeet\OneDrive\Desktop\new-number-plate-detection\Automatic-License-Plate-Detection-main\Automatic-License-Plate-Detection-main\images\N61.jpeg"
image = load_img(path) 
image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get the normalized output

# Size of the orginal image
h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)


Height of the image = 457
Width of the image = 500


In [60]:
fig = px.imshow(image)
fig.update_layout(width=700, height=500,  margin=dict(l=10, r=10, b=10, t=10), xaxis_title='Figure 13 - TEST Image')

In [61]:
image_arr_224.shape

(224, 224, 3)

In [62]:
test_arr = image_arr_224.reshape(1,224,224,3)
test_arr.shape

(1, 224, 224, 3)

In [63]:
# Make predictions
coords = model.predict(test_arr)
coords

1/1 [==============================] - 2s 2s/step


array([[0.3115304 , 0.6819162 , 0.5590513 , 0.67040205]], dtype=float32)

In [64]:
# Denormalize the values
denorm = np.array([w,w,h,h])
coords = coords * denorm
coords

array([[155.76520562, 340.95808864, 255.48643279, 306.37373686]])

In [65]:
coords = coords.astype(np.int32)
coords

array([[155, 340, 255, 306]])

In [66]:
# Draw bounding on top the image
xmin, xmax,ymin,ymax = coords[0]
pt1 =(xmin,ymin)
pt2 =(xmax,ymax)
print(pt1, pt2)

(155, 255) (340, 306)


In [67]:
cv2.rectangle(image,pt1,pt2,(0,255,0),3)
fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10))

In [226]:
# Create pipeline
path = r"C:\Users\Sumeet\OneDrive\Desktop\new-number-plate-detection\Automatic-License-Plate-Detection-main\Automatic-License-Plate-Detection-main\images\N187.jpeg"
def object_detection(path):
    
    # Read image
    image = load_img(path) # PIL object
    image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
    image1 = load_img(path,target_size=(224,224))
    
    # Data preprocessing
    image_arr_224 = img_to_array(image1)/255.0 # Convert to array & normalized
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)
    
    # Make predictions
    coords = model.predict(test_arr)
    
    # Denormalize the values
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)
    
    # Draw bounding on top the image
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

image, cods = object_detection(path)

fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 14')

1/1 [==============================] - 0s 170ms/step
(572, 475) (841, 549)


In [227]:
#cropping the image
img = np.array(load_img(path))
xmin ,xmax,ymin,ymax = cods[0]
roi = img[ymin:ymax,xmin:xmax]
fig = px.imshow(roi)
fig.update_layout(width=350, height=250, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 15 Cropped image')

In [228]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"

# extract text from image
text = pt.image_to_string(roi)
print(text)

MH 15BD8877|



In [229]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'./images/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,images\N1.xml,1093,1396,645,727,./images/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,images\N100.xml,134,301,312,350,./images/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,images\N101.xml,31,139,128,161,./images/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,images\N102.xml,164,316,216,243,./images/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,images\N103.xml,813,1067,665,724,./images/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


In [230]:
# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,images\N1.xml,1093,1396,645,727,./images/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,images\N100.xml,134,301,312,350,./images/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,images\N101.xml,31,139,128,161,./images/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,images\N102.xml,164,316,216,243,./images/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,images\N103.xml,813,1067,665,724,./images/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167
